In [ ]:
import os
import cohere
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 환경 변수 로드
load_dotenv(override=True)

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# =========================================================
# 2. [전처리] 키워드 정규화 (Inline Injection 방식)
# =========================================================
def normalize_query(user_query):
    """
    사용자 쿼리 내의 일상 용어를 법률 용어로 변환하여,
    문맥 내에 '일상어(법률어)' 형태로 주입합니다.
    예: "집주인이 월세를 올려" -> "집주인(임대인)이 월세(차임)를 올려(증액)"
    """
    
    # 1. 마스터 키워드 딕셔너리 (긴 단어부터 매칭되도록 정렬 필요)
    # Key: 일상어, Value: 법률어
    keyword_dict = {
        # [주체]
        "집주인": "임대인", "주인": "임대인", "건물주": "임대인", "임대업자": "임대인",
        "세입자": "임차인", "월세입자": "임차인",
        "부동산": "공인중개사", "중개인": "공인중개사",
        
        # [금전]
        "보증금": "임차보증금", "전세금": "임차보증금", 
        "월세": "차임", "방세": "차임",
        "관리비": "관리비용",
        
        # [행위 - 돈]
        "올려": "증액", "인상": "증액", "더 달라고": "증액", "더 내라고": "증액",
        "깎아": "감액", "내려": "감액", "할인": "감액",
        "돌려주지": "반환", "안 줘": "반환거부", "떼먹": "미반환",
        
        # [행위 - 거주/계약]
        "계약서": "임대차계약증서", "특약": "특약사항",
        "이사": "주택의인도", "전입": "주민등록/대항력",
        "살고": "점유", "거주": "점유",
        "나가라": "계약갱신거절/명도", "나가라고": "계약갱신거절/명도", "비워": "명도", "쫓겨": "명도/대항력",
        "방 빼": "계약해지", "나갈게": "계약해지", "해지": "계약해지",
        "연장": "계약갱신", "더 살": "계약갱신", "재계약": "계약갱신",
        "수리": "수선의무/필요비", "고쳐": "수선의무", "물 새": "누수", "곰팡이": "하자",
        "청소비": "원상회복비용", "원상복구": "원상회복"
    }
    
    normalized = user_query
    
    # 2. 치환 로직 (단어가 겹칠 경우 긴 단어 우선 매칭을 위해 키 길이로 정렬)
    # 예: '전세금'이 '전세'보다 먼저 처리되어야 함
    sorted_keys = sorted(keyword_dict.keys(), key=len, reverse=True)
    
    for key in sorted_keys:
        if key in normalized:
            legal_term = keyword_dict[key]
            # 이미 변환된 부분(괄호 안)은 건드리지 않도록 체크할 수도 있으나,
            # 간단하게는 replace를 사용하되, 무한 루프 방지를 위해 한 번만 실행하거나
            # "집주인" -> "집주인(임대인)"으로 바꾸면 다음 루프에서 "주인"이 또 매칭될 수 있음.
            # 이를 방지하기 위해, 여기서는 단순 replace를 쓰되
            # 이미 (법률어)가 붙어있는지 확인하는 로직은 복잡해지므로
            # 가장 직관적인 '단순 치환' 후 '중복 정제'는 LLM에게 맡깁니다.
            
            # 핵심: "단어" -> "단어(법률용어)" 형태로 변환
            # 다만, 이미 변환된 단어 내부의 글자가 또 매칭되는 것을 막기 위해 임시 토큰을 쓸 수도 있음.
            # 여기서는 파트너님의 이해를 돕기 위해 직관적으로 처리합니다.
            
            # "주인"이 "집주인"의 일부로 매칭되는 것을 방지하기 위해
            # 해당 key가 다른 key의 부분집합이 아닌 경우만 처리하거나,
            # replace 시 조심해야 합니다. 
            
            # 안전한 방법: 문장을 훼손하지 않는 선에서 괄호 병기
            target = f"{key}({legal_term})"
            if target not in normalized: # 이미 변환된 게 아니면
                normalized = normalized.replace(key, target)
                
    return normalized

# =========================================================
# 3. [검색 & Rerank] 하이브리드 검색 (변경 없음)
# =========================================================
def legal_hybrid_retrieval(query, pinecone_store, k_fetch=30, k_final=10, score_threshold=0.25):
    print(f"🔍 [1차 검색] Pinecone 탐색: '{query}'")
    try:
        initial_docs = pinecone_store.similarity_search(query, k=k_fetch)
    except Exception as e:
        print(f"❌ 검색 실패: {e}")
        return []
    
    if not initial_docs:
        return []

    # Cohere Rerank
    docs_content = [doc.page_content for doc in initial_docs]
    co = cohere.Client(api_key=COHERE_API_KEY)
    
    try:
        rerank_results = co.rerank(
            model="rerank-multilingual-v3.0",
            query=query, # 정규화된 쿼리 그대로 사용
            documents=docs_content,
            top_n=k_fetch
        )
    except Exception as e:
        print(f"⚠️ Rerank 실패: {e}")
        return initial_docs[:k_final]

    relevant_docs = []
    for r in rerank_results.results:
        if r.relevance_score < score_threshold:
            continue
        doc = initial_docs[r.index]
        doc.metadata['relevance_score'] = r.relevance_score
        relevant_docs.append(doc)

    final_docs = sorted(
        relevant_docs, 
        key=lambda x: int(x.metadata.get('priority', 99))
    )
    
    return final_docs[:k_final]

# =========================================================
# 4. [포맷팅]
# =========================================================
def format_legal_docs(docs):
    display_html = '<div class="legal-docs-container mb-3">'
    llm_context = ""
    
    for i, doc in enumerate(docs):
        src_title = doc.metadata.get('src_title', '법령')
        title = doc.metadata.get('title', '')
        
        # 원본의 더블 엔터(\n\n)를 싱글 엔터(\n)로 치환
        content = doc.page_content.strip().replace("\n\n", "\n")
        
        priority = doc.metadata.get('priority', 99)
        
        # LLM 입력용 (가독성을 위해 조항 사이에는 \n\n 유지)
        llm_context += f"참고문서 {i+1}.\n출처: {src_title}\n조항: {title}\n내용: {content}\n\n"
        
        # HTML용
        badge_color = "bg-danger" if priority == 1 else "bg-secondary"
        display_html += f"""
        <div class="card mb-2 shadow-sm">
            <div class="card-header d-flex justify-content-between align-items-center py-2">
                <span class="fw-bold text-dark">{title}</span>
                <span class="badge {badge_color}">{src_title}</span>
            </div>
            <div class="card-body py-2 bg-light">
                <p class="card-text text-secondary" style="white-space: pre-line; font-size: 0.9rem;">{content}</p>
            </div>
        </div>
        """
    display_html += '</div>'
    
    return {"display_html": display_html, "llm_context": llm_context}

# =========================================================
# 5. [생성] LLM 답변 생성 (프롬프트 강화)
# =========================================================
def generate_legal_advice(normalized_query, context_text):
    print(f"🤖 [LLM 입력] Query: {normalized_query}")
    
    system_prompt = """
    당신은 대한민국 부동산 법률(주택임대차보호법, 민법 등)에 정통한 'AI 법률 비서'입니다.
    사용자 질문에는 법률적 맥락을 돕기 위해 '(법률용어)'가 병기되어 있을 수 있습니다. 이를 참고하여 정확하게 분석하세요.

    [지시사항]
    1. **근거 기반**: 반드시 [참고 법령]에 기반하여 답변하세요.
    2. **입력 분석**: 질문에 포함된 괄호 안의 법률 용어(예: 명도, 대항력 등)를 핵심 키워드로 활용하세요.
    3. **답변 구조**:
       - 결론부터 명확히(가능/불가능) 제시
       - 법적 근거(제N조) 상세 설명
       - 사용자 상황에 맞는 조언
    4. **🚨 위험 경고**: 불리한 특약이나 법 위반 사항(강행규정 위반)은 강력하게 경고하세요.
    5. **면책**: 법적 효력이 없음을 밝히세요.

    [참고 법령]
    {context}
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "사용자 입력: {question}"),
    ])

    # 모델 설정
    llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.1) 

    chain = prompt | llm | StrOutputParser()
    
    try:
        return chain.invoke({"context": context_text, "question": normalized_query})
    except Exception as e:
        return f"❌ 오류: {str(e)}"

c:\Users\Admin\Desktop\Project2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# =========================================================
# [Main Execution Block]
# =========================================================

# 1. Pinecone DB 연결 (최초 1회 설정)
embedding_model = UpstageEmbeddings(model="solar-embedding-1-large-passage")
vector_store = PineconeVectorStore(
    pinecone_api_key=PINECONE_API_KEY,
    index_name="law-index",
    embedding=embedding_model
)

In [3]:
# ---------------------------------------------------------
# ▼▼▼ 테스트할 질문을 여기에 입력하세요 ▼▼▼
user_input = "집주인이 월세를 올려 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 계약서에는 그런 얘기가 없는데 이렇게 쫓겨나도 되나요?"
# user_input = "특약사항: 퇴실 시 청소비 30만원을 무조건 공제한다."
# ---------------------------------------------------------

print(f"📢 [Step 1] 사용자 질문: {user_input}")

# 2. 정규화 (Normalization)
norm_query = normalize_query(user_input)
print(f"🔄 [Step 2] 검색용 쿼리 변환: {norm_query}")

# 3. 검색 및 Reranking
retrieved_docs = legal_hybrid_retrieval(
    query=norm_query, 
    pinecone_store=vector_store,
    k_fetch=30,  # 1차 후보
    k_final=5,   # 최종 선별
    score_threshold=0.25
)

if retrieved_docs:
    # 4. 포맷팅
    formatted_data = format_legal_docs(retrieved_docs)
    
    # (선택) 검색된 조항 미리보기
    print(f"\n📚 [Step 3] 최종 선별된 문서 {len(retrieved_docs)}개:")
    print(formatted_data['llm_context'])
    print("-" * 50)
    
    # 5. LLM 답변 생성
    final_answer = generate_legal_advice(
        normalized_query=user_input, # LLM에게는 원본 질문을 주는 게 자연스러울 수 있음 (또는 norm_query)
        context_text=formatted_data['llm_context']
    )
    
    print("=" * 50)
    print("🤖 [Step 4] AI 최종 답변")
    print("=" * 50)
    print(final_answer)

    # (웹 개발용) HTML 코드 확인
    # print("\n[Web Display HTML]")
    # print(formatted_data['display_html'])

else:
    print("⚠️ 관련 법령을 찾을 수 없습니다. (점수 미달)")

📢 [Step 1] 사용자 질문: 집주인이 월세를 올려 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 계약서에는 그런 얘기가 없는데 이렇게 쫓겨나도 되나요?
🔄 [Step 2] 검색용 쿼리 변환: 집주인(임대인)이 월세(차임)를 올려(증액) 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라(계약갱신거절/명도)고(계약갱신거절/명도) 하네요. 계약서(임대차계약증서)에는 그런 얘기가 없는데 이렇게 쫓겨(명도/대항력)나도 되나요?
🔍 [1차 검색] Pinecone 탐색: '집주인(임대인)이 월세(차임)를 올려(증액) 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라(계약갱신거절/명도)고(계약갱신거절/명도) 하네요. 계약서(임대차계약증서)에는 그런 얘기가 없는데 이렇게 쫓겨(명도/대항력)나도 되나요?'

📚 [Step 3] 최종 선별된 문서 4개:
참고문서 1.
출처: 주택임대차보호법(법률)(제21065호)(20260102)
조항: 제7조 차임 및 보증금 증감청구권
내용: 제7조 (차임 등의 증감청구권) ① 당사자는 약정한 차임이나 보증금이 임차주택에 관한 조세, 공과금, 그 밖의 부담의 증감이나 경제사정의 변동으로 인하여 적절하지 아니하게 된 때에는 장래에 대하여 그 증감을 청구할 수 있다. 이 경우 증액청구는 임대차계약 또는 약정한 차임이나 보증금의 증액이 있은 후 1년 이내에는 하지 못한다. <개정 2020. 7. 31.>
② 제1항에 따른 증액청구는 약정한 차임이나 보증금의 20분의 1의 금액을 초과하지 못한다. 다만, 특별시ㆍ광역시ㆍ특별자치시ㆍ도 및 특별자치도는 관할 구역 내의 지역별 임대차 시장 여건 등을 고려하여 본문의 범위에서 증액청구의 상한을 조례로 달리 정할 수 있다.<신설 2020. 7. 31.>
[전문개정 2008. 3. 21.]

참고문서 2.
출처: 주택임대차보호법(법률)(제21065호)(20260102)
조항: 제10조 초과 차임 반환 청구 규정
내용: 제10조 의2(초과 차임 등의 반환청구) 임차인이 제7

In [4]:
# ---------------------------------------------------------
# ▼▼▼ 테스트할 질문을 여기에 입력하세요 ▼▼▼
# user_input = "집주인이 월세를 올려 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 계약서에는 그런 얘기가 없는데 이렇게 쫓겨나도 되나요?"
user_input = "계약서의 이 조항 괜찮을까? '특약사항: 임차인은 퇴실 시 청소비 30만원을 무조건 공제한다.'"
# ---------------------------------------------------------

print(f"📢 [Step 1] 사용자 질문: {user_input}")

# 2. 정규화 (Normalization)
norm_query = normalize_query(user_input)
print(f"🔄 [Step 2] 검색용 쿼리 변환: {norm_query}")

# 3. 검색 및 Reranking
retrieved_docs = legal_hybrid_retrieval(
    query=norm_query, 
    pinecone_store=vector_store,
    k_fetch=30,  # 1차 후보
    k_final=5,   # 최종 선별
    score_threshold=0.25
)

if retrieved_docs:
    # 4. 포맷팅
    formatted_data = format_legal_docs(retrieved_docs)
    
    # (선택) 검색된 조항 미리보기
    print(f"\n📚 [Step 3] 최종 선별된 문서 {len(retrieved_docs)}개:")
    print(formatted_data['llm_context'])
    print("-" * 50)
    
    # 5. LLM 답변 생성
    final_answer = generate_legal_advice(
        normalized_query=user_input, # LLM에게는 원본 질문을 주는 게 자연스러울 수 있음 (또는 norm_query)
        context_text=formatted_data['llm_context']
    )
    
    print("=" * 50)
    print("🤖 [Step 4] AI 최종 답변")
    print("=" * 50)
    print(final_answer)

    # (웹 개발용) HTML 코드 확인
    # print("\n[Web Display HTML]")
    # print(formatted_data['display_html'])

else:
    print("⚠️ 관련 법령을 찾을 수 없습니다. (점수 미달)")

📢 [Step 1] 사용자 질문: 계약서의 이 조항 괜찮을까? '특약사항: 임차인은 퇴실 시 청소비 30만원을 무조건 공제한다.'
🔄 [Step 2] 검색용 쿼리 변환: 계약서(임대차계약증서)의 이 조항 괜찮을까? '특약(특약사항)사항: 임차인은 퇴실 시 청소비(원상회복비용) 30만원을 무조건 공제한다.'
🔍 [1차 검색] Pinecone 탐색: '계약서(임대차계약증서)의 이 조항 괜찮을까? '특약(특약사항)사항: 임차인은 퇴실 시 청소비(원상회복비용) 30만원을 무조건 공제한다.''

📚 [Step 3] 최종 선별된 문서 2개:
참고문서 1.
출처: 주택임대차보호법(법률)(제21065호)(20260102)
조항: 제3조 **보증금 회수 및 우선변제권**
내용: 제3조 의2(보증금의 회수) ① 임차인(제3조제2항 및 제3항의 법인을 포함한다. 이하 같다)이 임차주택에 대하여 보증금반환청구소송의 확정판결이나 그 밖에 이에 준하는 집행권원(執行權原)에 따라서 경매를 신청하는 경우에는 집행개시(執行開始)요건에 관한 「민사집행법」 제41조에도 불구하고 반대의무(反對義務)의 이행이나 이행의 제공을 집행개시의 요건으로 하지 아니한다. <개정 2013. 8. 13.>
② 제3조제1항ㆍ제2항 또는 제3항의 대항요건(對抗要件)과 임대차계약증서(제3조제2항 및 제3항의 경우에는 법인과 임대인 사이의 임대차계약증서를 말한다)상의 확정일자(確定日字)를 갖춘 임차인은 「민사집행법」에 따른 경매 또는 「국세징수법」에 따른 공매(公賣)를 할 때에 임차주택(대지를 포함한다)의 환가대금(換價代金)에서 후순위권리자(後順位權利者)나 그 밖의 채권자보다 우선하여 보증금을 변제(辨濟)받을 권리가 있다.<개정 2013. 8. 13.>
③ 임차인은 임차주택을 양수인에게 인도하지 아니하면 제2항에 따른 보증금을 받을 수 없다.
④ 제2항 또는 제7항에 따른 우선변제의 순위와 보증금에 대하여 이의가 있는 이해관계인은 경매법원이나 체납처분청에 이의를 신청할 수 있다.<개정 2013. 8. 13.>
⑤ 제4

In [5]:
import os
import cohere
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 환경 변수 로드
load_dotenv(override=True)

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# =========================================================
# 2. [전처리] 키워드 정규화 (Inline Injection 방식)
# =========================================================
def normalize_query(user_query):
    """
    사용자 쿼리 내의 일상 용어를 법률 용어로 변환하여,
    문맥 내에 '일상어(법률어)' 형태로 주입합니다.
    예: "집주인이 월세를 올려" -> "집주인(임대인)이 월세(차임)를 올려(증액)"
    """
    
    # 1. 마스터 키워드 딕셔너리 (긴 단어부터 매칭되도록 정렬 필요)
    # Key: 일상어, Value: 법률어
    keyword_dict = {
        # [주체]
        "집주인": "임대인", "주인": "임대인", "건물주": "임대인", "임대업자": "임대인",
        "세입자": "임차인", "월세입자": "임차인",
        "부동산": "공인중개사", "중개인": "공인중개사",
        
        # [금전]
        "보증금": "임차보증금", "전세금": "임차보증금", 
        "월세": "차임", "방세": "차임",
        "관리비": "관리비용",
        
        # [행위 - 돈]
        "올려": "증액", "인상": "증액", "더 달라고": "증액", "더 내라고": "증액",
        "깎아": "감액", "내려": "감액", "할인": "감액",
        "돌려주지": "반환", "안 줘": "반환거부", "떼먹": "미반환",
        
        # [행위 - 거주/계약]
        "계약서": "임대차계약증서", "특약": "특약사항",
        "이사": "주택의인도", "전입": "주민등록/대항력",
        "살고": "점유", "거주": "점유",
        "나가라": "계약갱신거절/명도", "나가라고": "계약갱신거절/명도", "비워": "명도", "쫓겨": "명도/대항력",
        "방 빼": "계약해지", "나갈게": "계약해지", "해지": "계약해지",
        "연장": "계약갱신", "더 살": "계약갱신", "재계약": "계약갱신",
        "수리": "수선의무/필요비", "고쳐": "수선의무", "물 새": "누수", "곰팡이": "하자",
        "청소비": "원상회복비용", "원상복구": "원상회복"
    }
    
    normalized = user_query
    
    # 2. 치환 로직 (단어가 겹칠 경우 긴 단어 우선 매칭을 위해 키 길이로 정렬)
    # 예: '전세금'이 '전세'보다 먼저 처리되어야 함
    sorted_keys = sorted(keyword_dict.keys(), key=len, reverse=True)
    
    for key in sorted_keys:
        if key in normalized:
            legal_term = keyword_dict[key]
            # 이미 변환된 부분(괄호 안)은 건드리지 않도록 체크할 수도 있으나,
            # 간단하게는 replace를 사용하되, 무한 루프 방지를 위해 한 번만 실행하거나
            # "집주인" -> "집주인(임대인)"으로 바꾸면 다음 루프에서 "주인"이 또 매칭될 수 있음.
            # 이를 방지하기 위해, 여기서는 단순 replace를 쓰되
            # 이미 (법률어)가 붙어있는지 확인하는 로직은 복잡해지므로
            # 가장 직관적인 '단순 치환' 후 '중복 정제'는 LLM에게 맡깁니다.
            
            # 핵심: "단어" -> "단어(법률용어)" 형태로 변환
            # 다만, 이미 변환된 단어 내부의 글자가 또 매칭되는 것을 막기 위해 임시 토큰을 쓸 수도 있음.
            # 여기서는 파트너님의 이해를 돕기 위해 직관적으로 처리합니다.
            
            # "주인"이 "집주인"의 일부로 매칭되는 것을 방지하기 위해
            # 해당 key가 다른 key의 부분집합이 아닌 경우만 처리하거나,
            # replace 시 조심해야 합니다. 
            
            # 안전한 방법: 문장을 훼손하지 않는 선에서 괄호 병기
            target = f"{key}({legal_term})"
            if target not in normalized: # 이미 변환된 게 아니면
                normalized = normalized.replace(key, target)
                
    return normalized

# =========================================================
# 3. [검색 & Rerank] 하이브리드 검색 (변경 없음)
# =========================================================
def legal_hybrid_retrieval(query, pinecone_store, k_fetch=30, k_final=10, score_threshold=0.25):
    print(f"🔍 [1차 검색] Pinecone 탐색: '{query}'")
    try:
        initial_docs = pinecone_store.similarity_search(query, k=k_fetch)
    except Exception as e:
        print(f"❌ 검색 실패: {e}")
        return []
    
    if not initial_docs:
        return []

    # Cohere Rerank
    docs_content = [doc.page_content for doc in initial_docs]
    co = cohere.Client(api_key=COHERE_API_KEY)
    
    try:
        rerank_results = co.rerank(
            model="rerank-multilingual-v3.0",
            query=query, # 정규화된 쿼리 그대로 사용
            documents=docs_content,
            top_n=k_fetch
        )
    except Exception as e:
        print(f"⚠️ Rerank 실패: {e}")
        return initial_docs[:k_final]

    relevant_docs = []
    for r in rerank_results.results:
        if r.relevance_score < score_threshold:
            continue
        doc = initial_docs[r.index]
        doc.metadata['relevance_score'] = r.relevance_score
        relevant_docs.append(doc)

    final_docs = sorted(
        relevant_docs, 
        key=lambda x: int(x.metadata.get('priority', 99))
    )
    
    return final_docs[:k_final]

# =========================================================
# 4. [포맷팅]
# =========================================================
def format_legal_docs(docs):
    display_html = '<div class="legal-docs-container mb-3">'
    llm_context = ""
    
    for i, doc in enumerate(docs):
        src_title = doc.metadata.get('src_title', '법령')
        title = doc.metadata.get('title', '')
        
        # 원본의 더블 엔터(\n\n)를 싱글 엔터(\n)로 치환
        content = doc.page_content.strip().replace("\n\n", "\n")
        
        priority = doc.metadata.get('priority', 99)
        
        # LLM 입력용 (가독성을 위해 조항 사이에는 \n\n 유지)
        llm_context += f"참고문서 {i+1}.\n출처: {src_title}\n조항: {title}\n내용: {content}\n\n"
        
        # HTML용
        badge_color = "bg-danger" if priority == 1 else "bg-secondary"
        display_html += f"""
        <div class="card mb-2 shadow-sm">
            <div class="card-header d-flex justify-content-between align-items-center py-2">
                <span class="fw-bold text-dark">{title}</span>
                <span class="badge {badge_color}">{src_title}</span>
            </div>
            <div class="card-body py-2 bg-light">
                <p class="card-text text-secondary" style="white-space: pre-line; font-size: 0.9rem;">{content}</p>
            </div>
        </div>
        """
    display_html += '</div>'
    
    return {"display_html": display_html, "llm_context": llm_context}

# =========================================================
# 5. [생성] LLM 답변 생성 (프롬프트 강화)
# =========================================================
def generate_legal_advice(normalized_query, context_text):
    print(f"🤖 [LLM 입력] Query: {normalized_query}")
    
    system_prompt = """
    당신은 대한민국 부동산 법률(주택임대차보호법, 민법 등)에 정통한 'AI 법률 비서'입니다.
    사용자 질문에는 법률적 맥락을 돕기 위해 '(법률용어)'가 병기되어 있을 수 있습니다. 이를 참고하여 정확하게 분석하세요.

    [지시사항]
    1. **근거 기반**: 반드시 [참고 법령]에 기반하여 답변하세요.
    2. **입력 분석**: 질문에 포함된 괄호 안의 법률 용어(예: 명도, 대항력 등)를 핵심 키워드로 활용하세요.
    3. **답변 구조**:
       - 결론부터 명확히(가능/불가능) 제시
       - 법적 근거(제N조) 상세 설명
       - 사용자 상황에 맞는 조언
    4. **🚨 위험 경고**: 불리한 특약이나 법 위반 사항(강행규정 위반)은 강력하게 경고하세요.
    5. **면책**: 법적 효력이 없음을 밝히세요.

    [참고 법령]
    {context}
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "사용자 입력: {question}"),
    ])

    # 모델 설정
    llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.1) 

    chain = prompt | llm | StrOutputParser()
    
    try:
        return chain.invoke({"context": context_text, "question": normalized_query})
    except Exception as e:
        return f"❌ 오류: {str(e)}"

In [6]:
# ---------------------------------------------------------
# ▼▼▼ 테스트할 질문을 여기에 입력하세요 ▼▼▼
# user_input = "집주인이 월세를 올려 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 계약서에는 그런 얘기가 없는데 이렇게 쫓겨나도 되나요?"
user_input = "계약서의 이 조항 괜찮을까? '특약사항: 임차인은 퇴실 시 청소비 30만원을 무조건 공제한다.'"
# ---------------------------------------------------------

print(f"📢 [Step 1] 사용자 질문: {user_input}")

# 2. 정규화 (Normalization)
norm_query = normalize_query(user_input)
print(f"🔄 [Step 2] 검색용 쿼리 변환: {norm_query}")

# 3. 검색 및 Reranking
retrieved_docs = legal_hybrid_retrieval(
    query=norm_query, 
    pinecone_store=vector_store,
    k_fetch=30,  # 1차 후보
    k_final=5,   # 최종 선별
    score_threshold=0.25
)

if retrieved_docs:
    # 4. 포맷팅
    formatted_data = format_legal_docs(retrieved_docs)
    
    # (선택) 검색된 조항 미리보기
    print(f"\n📚 [Step 3] 최종 선별된 문서 {len(retrieved_docs)}개:")
    print(formatted_data['llm_context'])
    print("-" * 50)
    
    # 5. LLM 답변 생성
    final_answer = generate_legal_advice(
        normalized_query=user_input, # LLM에게는 원본 질문을 주는 게 자연스러울 수 있음 (또는 norm_query)
        context_text=formatted_data['llm_context']
    )
    
    print("=" * 50)
    print("🤖 [Step 4] AI 최종 답변")
    print("=" * 50)
    print(final_answer)

    # (웹 개발용) HTML 코드 확인
    # print("\n[Web Display HTML]")
    # print(formatted_data['display_html'])

else:
    print("⚠️ 관련 법령을 찾을 수 없습니다. (점수 미달)")

📢 [Step 1] 사용자 질문: 계약서의 이 조항 괜찮을까? '특약사항: 임차인은 퇴실 시 청소비 30만원을 무조건 공제한다.'
🔄 [Step 2] 검색용 쿼리 변환: 계약서(임대차계약증서)의 이 조항 괜찮을까? '특약(특약사항)사항: 임차인은 퇴실 시 청소비(원상회복비용) 30만원을 무조건 공제한다.'
🔍 [1차 검색] Pinecone 탐색: '계약서(임대차계약증서)의 이 조항 괜찮을까? '특약(특약사항)사항: 임차인은 퇴실 시 청소비(원상회복비용) 30만원을 무조건 공제한다.''

📚 [Step 3] 최종 선별된 문서 2개:
참고문서 1.
출처: 주택임대차보호법(법률)(제21065호)(20260102)
조항: 제3조 **보증금 회수 및 우선변제권**
내용: 제3조 의2(보증금의 회수) ① 임차인(제3조제2항 및 제3항의 법인을 포함한다. 이하 같다)이 임차주택에 대하여 보증금반환청구소송의 확정판결이나 그 밖에 이에 준하는 집행권원(執行權原)에 따라서 경매를 신청하는 경우에는 집행개시(執行開始)요건에 관한 「민사집행법」 제41조에도 불구하고 반대의무(反對義務)의 이행이나 이행의 제공을 집행개시의 요건으로 하지 아니한다. <개정 2013. 8. 13.>
② 제3조제1항ㆍ제2항 또는 제3항의 대항요건(對抗要件)과 임대차계약증서(제3조제2항 및 제3항의 경우에는 법인과 임대인 사이의 임대차계약증서를 말한다)상의 확정일자(確定日字)를 갖춘 임차인은 「민사집행법」에 따른 경매 또는 「국세징수법」에 따른 공매(公賣)를 할 때에 임차주택(대지를 포함한다)의 환가대금(換價代金)에서 후순위권리자(後順位權利者)나 그 밖의 채권자보다 우선하여 보증금을 변제(辨濟)받을 권리가 있다.<개정 2013. 8. 13.>
③ 임차인은 임차주택을 양수인에게 인도하지 아니하면 제2항에 따른 보증금을 받을 수 없다.
④ 제2항 또는 제7항에 따른 우선변제의 순위와 보증금에 대하여 이의가 있는 이해관계인은 경매법원이나 체납처분청에 이의를 신청할 수 있다.<개정 2013. 8. 13.>
⑤ 제4